# Double Well 3D

In [ ]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# System parameters.

n_particles = 1
mass = 35.453 * unit.amu # mass of Cl atom

In [ ]:
# Simulation parameters.

## Thermodynamic state and Langevin Dynamics

temperature = 300*unit.kelvin
friction = 1.0/unit.picosecond

## Times

simulation_time = 5.0*unit.nanosecond
saving_time = 1.0*unit.picoseconds
integration_timestep = 0.01*unit.picoseconds

In [ ]:
# System

system = mm.System()

for ii in range(n_particles):
    system.addParticle(mass)

In [ ]:
# External potential

Eo=3.0 * unit.kilocalories_per_mole
a=5.0 * unit.angstroms
b=0.0 * unit.kilocalories_per_mole
k=1.0 * unit.kilocalories_per_mole/unit.angstrom**2

force = mm.CustomExternalForce('A*x^4+B*x^2+C*x + D*(y^2+z^2)')
force.addGlobalParameter('A', Eo/(a**4))
force.addGlobalParameter('B', -2.0*Eo/(a**2))
force.addGlobalParameter('C', -b/a)
force.addGlobalParameter('D', k/2.0)

for ii in range(n_particles):
    force.addParticle(ii, [])
_ = system.addForce(force)

In [ ]:
# Integrator

integrator = mm.LangevinIntegrator(temperature, friction, integration_timestep)

In [ ]:
# Computing platform

platform_name = 'CPU' #'CPU' or 'CUDA'(Nvidia GPU) or 'OPENCL' (GPU not Nvidia)
platform = mm.Platform.getPlatformByName(platform_name)

In [ ]:
# Context

context = mm.Context(system, integrator, platform)

In [ ]:
# Initial conditions

initial_positions  = np.zeros([n_particles, 3], np.float32) * unit.angstroms
initial_velocities = np.zeros([n_particles, 3], np.float32) * unit.angstroms/unit.picoseconds

initial_positions[0,0] = a

context.setPositions(initial_positions)
context.setVelocities(initial_velocities)

In [ ]:
# Auxiliary simulation parameters
n_steps_per_period = int(saving_time/integration_timestep)
n_periods = int(simulation_time/saving_time)

In [ ]:
# Arrays to store times and positions

times = np.zeros([n_periods], np.float32) * unit.picoseconds
positions = np.zeros([n_periods, n_particles, 3], np.float32) * unit.angstroms

In [ ]:
# Time and positions for time 0

state = context.getState(getPositions=True)
times[0] = state.getTime()
positions[0] = state.getPositions()

In [ ]:
# Loop to run the simulation

for ii in tqdm(range(1, n_periods)):
    context.getIntegrator().step(n_steps_per_period)
    state = context.getState(getPositions=True, getVelocities=True, getEnergy=True)
    times[ii] = state.getTime()
    positions[ii] = state.getPositions()

In [ ]:
plt.rcParams['figure.figsize'] = 18, 4

for ii, axis_label in zip(range(3),['X','Y','Z']):
    plt.plot(times, positions[:,0,ii])
    plt.ylabel('{} ({})'.format(axis_label, positions.unit))
    plt.xlabel('time ({})'.format(times.unit))
    plt.show()